# Datenimport

Es wird bisher mehr importiert als gebraucht wird

In [1]:
import pandas as pd
import json

filepath = "/Users/david/dev_projects/chase-rank/data/processed/"
track = "2022-10-23T12-20-03_test_track/"

In [2]:
points_df = pd.read_csv(filepath+track+"points.csv")
section_main_dataframe = pd.read_csv(filepath+track+"sections.csv")
track_info = json.load(open(filepath+track+"track_info.json"))

In [3]:
import pandas as pd
import json
from pathlib import Path

Trackname = "2022-10-23T12-20-03_test_track"
Trackdate = """P"""
ProcessedTrackName = Trackdate+"_"+Trackname


DATA_PATH = Path("../data")
#TRACK_PATH = Path(DATA_PATH, "routes",Trackname,".gpx")
def get_ana_track_path(Trackname):
    ANA_TRACK_PATH = Path(DATA_PATH, "processed",Trackname)
    return ANA_TRACK_PATH

In [4]:
def get_points_df(ANA_TRACK_PATH):
    points_df = pd.read_csv(Path(ANA_TRACK_PATH,"points.csv"))
    return points_df

def get_section_df(ANA_TRACK_PATH):
    section_main_dataframe = pd.read_csv(Path(ANA_TRACK_PATH,"sections.csv"))
    return section_df

def get_track_info(ANA_TRACK_PATH):
    track_info = json.load(open(Path(ANA_TRACK_PATH,"track_info.json")))
    return track_info

# Pandas im Datenbankformat

In [5]:
db_strava_activities = "/Users/david/dev_projects/strava_statistics/strava_activities.csv"

db_strava = pd.read_csv(db_strava_activities)
db_strava = db_strava.drop("Unnamed: 0", axis=1)

In [6]:
strava_list=[]
user = "David"

for i in range(0,len(db_strava)):
    strava_datapoint = {
        "strava_id": db_strava["id"][i],#
        "strava_name": db_strava["name"][i],#
        "user": user,
        "distance": db_strava["distance"][i],#
        "moving_time": db_strava["moving_time"][i],#
        "elapsed_time": db_strava["elapsed_time"][i],#
        "total_elevation_gain": db_strava["total_elevation_gain"][i],#
        "sport_type": db_strava["type"][i],#
        "start_date": db_strava["start_date_local"][i],#
        "timezone": None,
        "start_lat": None,
        "start_lng": None,
        "end_lat": None,
        "end_lng": None,
        "average_speed": None,
        "max_speed": None,
        "elev_high": None,
        "elev_low": None,
        "external_id": db_strava["external_id"][i],#

        'surface_paved_smooth': None,
        'surface_compacted': None,
        'surface_path': None,
        'surface_paved': None,
        'surface_paved_rough': None,
        'surface_gravel': None,
        'surface_dirt': None,
        'surface_null': None,}

    strava_list.append(strava_datapoint)

In [7]:
main_dataframe = pd.DataFrame(strava_list)

# Monatsauswertung

Statistiken für die Honigmann Challenge auf Monatsebene

In [8]:
df = main_dataframe

# convert the datetime column to datetime type
main_dataframe['start_date'] = pd.to_datetime(df['start_date'])

# extract the month from the datetime column
df['year'] = df['start_date'].dt.year
df['month'] = df['start_date'].dt.month

all_users=[]
for user in main_dataframe["user"].unique():
    year_stat = []
    for year in range(2020,2023):
        year_df = df[df['year'] == year]

        for month in range(1,12):
            month_dataframe = year_df[year_df['month'] == month]

            month_stat = {
                "user":user,
                "year":year,
                "month":month,
                "activities":len(month_dataframe),
                "time_spent":month_dataframe["moving_time"].sum(),
                "distance":month_dataframe["distance"].sum(),
                "total_elevation_gain":month_dataframe["total_elevation_gain"].sum(),}


            year_stat.append(month_stat)
            del month_dataframe
    all_users.append(year_stat)


In [30]:
distance_L = []
elevation_L = []
year_month_L = []


for i in range(0,len(all_users[0])):
    year = all_users[0][i]['year']
    month = all_users[0][i]['month']

    year_month = str(year)+"_"+str(month)
    distance = all_users[0][i]['distance']
    distance = distance/1000

    elevation = all_users[0][i]['total_elevation_gain']

    distance_L.append(distance)
    year_month_L.append(year_month)
    elevation_L.append(elevation)


In [39]:
import plotly.express as px
fig = px.bar(y=distance_L, x=year_month_L, title="km per month")
fig.show()
fig = px.bar(y=elevation_L, x=year_month_L, title="elevation gain per month")
fig.show()